In [75]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import f

In [60]:
df = pd.read_csv('main.csv')

In [61]:
display(df)

,Page 1,Page 2,Page 3,Page 4
0,164,178,175,155
1,172,191,193,166
2,177,182,171,164
3,156,185,163,170
4,195,177,176,168



# 2. Descriptive Statistics

Compute the mean and variance for each group (web page).

Compute the overall mean for all time

In [62]:
media = df.mean()

In [63]:
varianza = df.var()

In [64]:
print("La media de cada columna es:")
print(media)

La media de cada columna es:
Page 1    172.8
Page 2    182.6
Page 3    175.6
Page 4    164.6
dtype: float64


In [65]:
print("La varianza de cada columna es:")
print(varianza)

La varianza de cada columna es:
Page 1    217.7
Page 2     32.3
Page 3    120.8
Page 4     33.8
dtype: float64


In [67]:
# Calcular la media global (gran media)
global_mean = stacked_data['Value'].mean()


# 3. Variance Decomposition:

Compute the total sum of squares (SST).

Compute the sum of squares between groups (SSB).

Compute the sum of squares within groups (SSW)

In [68]:
stacked_data = df.stack().reset_index()
stacked_data.columns = ['Index', 'Group', 'Value']

In [42]:
# 1. Total Sum of Squares (SST)
SST = ((stacked_data['Value'] - global_mean) ** 2).sum()

In [43]:
# 2. Sum of Squares Between Groups (SSB)
group_means = stacked_data.groupby('Group')['Value'].mean()
group_sizes = stacked_data.groupby('Group')['Value'].count()
SSB = ((group_means - global_mean) ** 2 * group_sizes).sum()

In [44]:
# 3. Sum of Squares Within Groups (SSW)
SSW = SST - SSB

In [49]:
# Resultados
print(f"Total Sum of Squares (SST): {SST}")
print(f"Sum of Squares Between Groups (SSB): {SSB}")
print(f"Sum of Squares Within Groups (SSW): {SSW}")

Total Sum of Squares (SST): 2449.7999999999997
Sum of Squares Between Groups (SSB): 831.3999999999999
Sum of Squares Within Groups (SSW): 1618.3999999999999


# 4. ANOVA Table Construction:

Compute degrees of freedom.

Obtain the between-group variance (MSB) and the within-group variance
(MSW).

Compute the F-statistic and compare it with the critical F-value from the F-table to
determine statistical significance

In [51]:
# Grados de libertad entre grupos (df_between)
k = len(df.columns)  # Número de grupos (columnas)
df_between = k - 1

# Grados de libertad dentro de grupos (df_within)
N = len(stacked_data)  # Número total de observaciones
df_within = N - k

# Grados de libertad totales (df_total)
df_total = N - 1

# 5. Varianzas (Mean Squares)
# Varianza entre grupos (MSB)
MSB = SSB / df_between

# Varianza dentro de grupos (MSW)
MSW = SSW / df_within



In [52]:

# 6. Estadístico F
F_statistic = MSB / MSW

# 7. Valor crítico de F (usando scipy.stats)
from scipy.stats import f

# Nivel de significancia (comúnmente 0.05)
alpha = 0.05

# Valor crítico de F
F_critical = f.ppf(1 - alpha, df_between, df_within)

# 8. Comparar el estadístico F con el valor crítico
if F_statistic > F_critical:
    print("Rechazamos la hipótesis nula: Hay diferencias significativas entre los grupos.")
else:
    print("No rechazamos la hipótesis nula: No hay diferencias significativas entre los grupos.")

No rechazamos la hipótesis nula: No hay diferencias significativas entre los grupos.


In [71]:
anova_results = {
    'Fuente de variación': ['Entre grupos (Between)', 'Dentro de grupos (Within)', 'Total'],
    'Suma de cuadrados': [SSB, SSW, SST],  # Suma de cuadrados
    'Grados de libertad': [df_between, df_within, df_total],  # Grados de libertad
    'MS': [MSB, MSW, None],  # Varianzas (Mean Squares)
    'F': [F_statistic, None, None]  # Estadístico F
}

In [73]:
anova_table = pd.DataFrame(anova_results)

In [74]:
display(anova_table)

,Fuente de variación,Suma de cuadrados,Grados de libertad,MS,F
0,Entre grupos (Between),831.4,3,277.133333,2.739825
1,Dentro de grupos (Within),1618.4,16,101.150000,NaN
2,Total,2449.8,19,NaN,NaN



# 5. P-value Calculation:

Obtain the p-value associated with the F-statistic.

Evaluate whether the p-value is smaller than the significance level ff=0.05 and decide
whether to reject

In [76]:
# Calcular el p-valor asociado al estadístico F
p_value = 1 - f.cdf(F_statistic, df_between, df_within)

In [80]:
# Evaluar si el p-valor es menor que el nivel de significancia
if p_value < alpha:
    print(f"Rechazamos la hipótesis nula: Hay diferencias significativas entre los grupos (p-valor = {p_value:.4f}).")
else:
    print(f"No rechazamos la hipótesis nula: No hay diferencias significativas entre los grupos (p-valor = {p_value:.4f}).")

No rechazamos la hipótesis nula: No hay diferencias significativas entre los grupos (p-valor = 0.0776).


In [81]:
# Agregar el p-valor al DataFrame de la tabla ANOVA
anova_table['p-value'] = [p_value, None, None]


In [83]:
print("\nTabla ANOVA con p-valor:")
display(anova_table)


Tabla ANOVA con p-valor:


,Fuente de variación,Suma de cuadrados,Grados de libertad,MS,F,p-value
0,Entre grupos (Between),831.4,3,277.133333,2.739825,0.077586
1,Dentro de grupos (Within),1618.4,16,101.150000,NaN,NaN
2,Total,2449.8,19,NaN,NaN,NaN


# 6. Conclusion and Interpretation of Result

Determine whether there are significant differences between the web pages in terms of
time spent.

Discuss possible implications of the results for web page optimization.

#### Based on the ANOVA table results:

The p-value is 0.077586, which is greater than the significance level (0.05). This means that there is not enough evidence to claim that there are significant differences in the time spent among the analyzed web pages.

Although there are some differences in the mean time spent on each page (as observed in the group means), these differences are not statistically significant at the 95% confidence level. This could be due to variability within the groups or the sample size.

#### Conclusion:

There are no significant differences in the time spent among the analyzed web pages.

#### Implications:

Although no significant differences were found, the results can help guide practical improvements in web page optimization. To achieve a more detailed analysis, increasing the sample size would be necessary to draw more solid conclusions.